### <span style="color:darkgreen"> Data analysis protocol (Excel and R)

---


1. Let’s open the shared table  in excel
<span style="color:blue">(BV5.trim.contigs.good.renamed.unique.pick.dgc.unique_list.0.03.abund.0.03.pick.0.03.pick.0.03.pick.0.03.pick.shared) </span>

 ![](pictures/excelAndR/1.png)

 <span style="color:red">Q: what is label? What is group and what is numOtus?

---
2. Let’s include the sample information in the excel table:

 ![](pictures/excelAndR/2.png)

3. Let’s plot the total number of reads per sample:
 ![](pictures/excelAndR/3.png)
 <span style="color:red">Q: what is the max number of reads per sample? What is the min?

4. Let’s see the number of reads per OTU in each sample (for the first 100 samples):
                                                         
 ![](pictures/excelAndR/4.png)                                                         
 
 <span style="color:red">Q: what are the most abundant OTUs in each of the samples? Use the taxonomy file to see the OTU id. Does it makes sense?

 <span style="color:red">Q: How can we normalize the data to compare the samples?
                                                   


---
5. Let’s export the data to work in R:
In R we need a sample information file (s.txt) and an OTU table (tab.txt). The rows have to match! Copy paste the data into a .txt file.

 ![](pictures/excelAndR/5.png)

 <span style="color:green">Note:(we already located the files in /home/ubuntu/scripts/data/)
    


---
6. Let’s load the samples and table:


In [ ]:
setwd('/home/ubuntu/scripts/') #set working directory

In [ ]:
tab <- read.table("data/tab.txt", h=T, sep='\t') #load the tab.txt file
tab #print the file

In [ ]:
s <- read.table("data/s.txt", h=T, sep='\t') ##load the s.txt file 
s #print the file

In [ ]:
s$Sample <- factor(s$sample_type, levels=c("At_leaf", "Soil", "Chicken_poo", "Fruit_fly"))
s

---
7. Let’s create the rarefaction curves to see how well are we capturing microbial diversity in the different samples:


In [ ]:
library(vegan) # load vegan packages 

In [ ]:
rarecurve(tab, step=100, col = c(rep("chartreuse3",3), rep("blue",3), rep( "red",3), rep("brown",3)), cex = 0.6,  xlab = "Sample Size", ylab = "Number of OTUs in all samples", label=F)

#Green=At_leaf, Blue= Soil, Red=Chicken_poo, Brown= Fruit_fly


 ![](pictures/excelAndR/6.png)


 <span style="color:red">Q: How is a rarefaction curve constructed? What is “step” in the command? What happens if step=10000?

 <span style="color:red">Q: What is the highest diversity sample? What is the lowest? Have we sequenced enough reads from each sample?


---

8. Let’s see how we can rarefy the samples. In theory it should be to the lowest number of reads in a sample. Here the lowest number of reads in a sample is 2964.
Let’s see what happens if we rarefy to 2964:


In [ ]:
rarecurve(tab, step=1000, sample=2964, col = c(rep("chartreuse3",3), rep("blue",3), rep( "red",3), rep("brown",3)), cex = 0.6,  xlab = "Sample Size", ylab = "Number of OTUs in all samples", label=F)

 ![](pictures/excelAndR/7.png)
 
<span style="color:red">Q: Do you think this would be a good idea? Would we be under estimating the diversity? By a lot?

---
9. Let’s rarefy the samples:

In [ ]:
rarefied.tab <- rrarefy(tab, sample=2964)


10. take a look at the new table. 

<span style="color:red">Q: How many reads for Otu10 are in which sample 10? 93?



In [ ]:
rarefied.tab[, "Otu00010"]

---

11.Let’s calculate 3 alpha diversity indexes in the samples to compare them:




In [ ]:
richness<-specnumber(rarefied.tab, MARGIN = 1)

In [ ]:
shannon<-diversity(rarefied.tab, index = "shannon", MARGIN = 1, base = exp(1))

In [ ]:
simpson<-diversity(rarefied.tab, index = "simpson", MARGIN = 1, base = exp(1))

In [ ]:
rarefied.s<-cbind(s,richness,shannon,simpson)

12. Let’s plot the results:

In [ ]:
par(mfrow=c(1,3))

plot(rarefied.s$richness~rarefied.s$sample_type)
plot(rarefied.s$shannon~rarefied.s$sample_type)
plot(rarefied.s$simpson~rarefied.s$sample_type)


 ![](pictures/excelAndR/8.png)
 
<span style="color:red"> Q: How are each of these indexes calculated? 
    
<span style="color:red">Q: The simpson and shannon indexes seems to give a bigger difference between chicken_poo and fruit_fly samples. Why is this i.e. what could be the explanation?


---
13. Ok let’s calculate beta-diversity indexes (distances) to compare the samples and visualize the differences by PCoA and hierarchical-clustering.

First with Euclidean distances:

In [ ]:
library(ade4)

#First with Euclidean distances:

vegdist(rarefied.tab, method="euclidean", binary=FALSE)
pca_euc_rarefied.tab<-dudi.pco(vegdist(rarefied.tab, method="euclidean", binary=FALSE), scannf=F)

s.class(pca_euc_rarefied.tab $li, s$Sample, xax=1, yax=2, cpoint=3, grid=F, addaxes=T, cellipse=1, cstar=0, axesell=0, col = c("chartreuse3","blue","red","brown"), clabel = 0.5, sub="Euclidean")



In [ ]:
#then with Bray-curtis distances (dissimilarity):

vegdist(rarefied.tab, method="bray", binary=FALSE)
pca_bray_rarefied.tab<-dudi.pco(vegdist(rarefied.tab, method="bray", binary=FALSE), scannf=F)

s.class(pca_bray_rarefied.tab $li, s$Sample, xax=1, yax=2, cpoint=3, grid=F, addaxes=T, cellipse=1, cstar=0, axesell=0, col = c("chartreuse3","blue","red","brown"), clabel = 0.5, sub="Bray-Curtis")


---
14. check the weight of each of the axes i.e. the percentage of variability captured by the axis:

In [ ]:
inertia(pca_euc_rarefied.tab)
inertia(pca_bray_rarefied.tab)

![](pictures/excelAndR/9.png)

In [ ]:
par(mfrow=c(1,2))

plot(hclust(vegdist(rarefied.tab, method="euclidean", binary=FALSE), method = "complete", members = NULL), labels=s$Sample)

plot(hclust(vegdist(rarefied.tab, method="bray", binary=FALSE), method = "complete", members = NULL), labels=s$Sample)


![](pictures/excelAndR/10.png)

<span style="color:red">Q: Do these two distance methods give similar results? What is the main difference?

---
15. Now let’s say that one of our samples didn’t work so well and we obtained only 1000 reads. In that case we would have to rarefy to 1000 reads.

rarefied.tab<-rrarefy(tab, sample=100)

<span style="color:red">Q: How does this affects our results? What changes in comparison to the previous rarefaction?

Run all the commands from step 11 and compare the results

In [ ]:
simper(tab, s$sample_type, permutations = 0, trace = FALSE)

<span style="color:red">Q: What does the output tell us? (https://rdrr.io/cran/vegan/man/simper.html)

Now run a significance test for individual OTUs to see if results are relevant

In [ ]:
kruskal.test(tab$Otu00002 ~ s$sample_type)

<span style="color:red">Q: What does the statistics tell us?

### Final short presentations:


<img src="pictures/excelAndR/presentationIcon.png"  width="60%" height="30%">

### describe your microbial community

### Remind us what is the sample origin.

### For both 18S and 16S describe…
1. Level of diversity (number of OTUs)
2. Difference in comparison to other samples? Which is the most similar?
3. What were the main “contaminant” i.e. non-microbial OTUs that had to be deleted? For this you need to look into the fisrt “shared” file created in mothur.
4. What percentage of the sample did they represent?

5. What are the main OTUs found in the sample? (max. 10 most abundant)
6. What is their phylogenetic classification?
7. To what type of organism do they correspond?
8. Is it normal or unusual to find those microbes in that type of sample?
9. What could they be doing there?
10. What would you do to find out what they are doing there (i.e. the function)? Briefly describe an experiment you would do.

---
### <span style="color:green">Hints: 
    
•	Get the representative sequences from the OTUs and blast them to see if you get a better identification.
    
•	Think about how 18S and 16S results could be connected.
    
•	Do not limit your literature search to Wikipedia! Go to google scholar (https://scholar.google.com) and get the real information from scientific papers.